In [1]:
!nvidia-smi


Tue May  7 17:20:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
%%cu
#include <iostream>

__global__ void helloCUDA()
{
    printf("Hello CUDA from GPU!\n");
}

int main()
{
    helloCUDA<<<1, 1>>>();
    cudaDeviceSynchronize();
    return 0;
}


UsageError: Cell magic `%%cu` not found.


In [ ]:
%%writefile cuda_code.cu

#include <iostream>

__global__ void helloCUDA()
{
    printf("Hello CUDA from GPU!\n");
}

int main()
{
    helloCUDA<<<1, 1>>>();
    cudaDeviceSynchronize();
    return 0;
}


Writing cuda_code.cu


In [ ]:
!nvcc -o cuda_code_cuda cuda_code.cu
!./cuda_code_cuda


Hello CUDA from GPU!


In [ ]:
%%writefile cuda_code_1.cu

#include <iostream>
#include <vector>
#include <cstdlib>
#include <cuda_runtime.h>
#include <cublas_v2.h>

int main() {
    const int batch_size = 128;
    const int input_size = 256;
    const int output_size = 10;  // 10 output classes for MNIST

    // Allocate host memory for input and weights
    std::vector<float> input(batch_size * input_size);
    std::vector<float> weights(input_size * output_size);

    // Initialize input and weights with random values
    for (int i = 0; i < batch_size * input_size; ++i) {
        input[i] = static_cast<float>(rand()) / RAND_MAX;
    }
    for (int i = 0; i < input_size * output_size; ++i) {
        weights[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Allocate device memory
    float *d_input, *d_weights, *d_output;
    cudaMalloc(&d_input, batch_size * input_size * sizeof(float));
    cudaMalloc(&d_weights, input_size * output_size * sizeof(float));
    cudaMalloc(&d_output, batch_size * output_size * sizeof(float));

    // Copy input and weights to device
    cudaMemcpy(d_input, input.data(), batch_size * input_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_weights, weights.data(), input_size * output_size * sizeof(float), cudaMemcpyHostToDevice);

    // Create cuBLAS handle
    cublasHandle_t handle;
    cublasCreate(&handle);

    // Perform vector multiplication
    float alpha = 1.0f, beta = 0.0f;
    cublasSgemv(handle, CUBLAS_OP_N, output_size, input_size, &alpha,
                d_weights, output_size, d_input, 1, &beta, d_output, 1);

    // Destroy cuBLAS handle
    cublasDestroy(handle);

    // Copy output back to host
    std::vector<float> output(batch_size * output_size);
    cudaMemcpy(output.data(), d_output, batch_size * output_size * sizeof(float), cudaMemcpyDeviceToHost);

    // Free device memory
    cudaFree(d_input);
    cudaFree(d_weights);
    cudaFree(d_output);

    // Print a sample output
    std::cout << "Sample output:" << std::endl;
    for (int i = 0; i < 10; ++i) {
        std::cout << output[i] << " ";
    }
    std::cout << std::endl;

    return 0;
}

Writing cuda_code_1.cu


In [ ]:
!nvcc -o cuda_code_1_cuda cuda_code_1.cu


/usr/bin/ld: /tmp/tmpxft_000008d5_00000000-11_cuda_code_1.o: in function `main':
tmpxft_000008d5_00000000-6_cuda_code_1.cudafe1.cpp:(.text+0x20a): undefined reference to `cublasCreate_v2'
/usr/bin/ld: tmpxft_000008d5_00000000-6_cuda_code_1.cudafe1.cpp:(.text+0x270): undefined reference to `cublasSgemv_v2'
/usr/bin/ld: tmpxft_000008d5_00000000-6_cuda_code_1.cudafe1.cpp:(.text+0x280): undefined reference to `cublasDestroy_v2'
collect2: error: ld returned 1 exit status


In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Use GPU 0

In [ ]:
!nvidia-smi

Mon Apr 29 20:06:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8               8W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!ldconfig /usr/local/cuda-11.8/targets/x86_64-linux/lib/


/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link



In [ ]:
!ldconfig /usr/local/cuda/targets/x86_64-linux/lib/


/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link



In [ ]:
%%cuda --compiler-options '-lcublas -lcudart -lcudnn -lcurand' --include-paths '/usr/local/cuda/targets/x86_64-linux/include/' --libraries '/usr/local/cuda/targets/x86_64-linux/lib/'

UsageError: Cell magic `%%cuda` not found.


In [ ]:
%%writefile cuda_code_1.cu

#include <iostream>
#include <vector>
#include <cstdlib>
#include <cuda_runtime.h>
#include <cublas_v2.h>

int main() {
    const int batch_size = 128;
    const int input_size = 256;
    const int output_size = 10;  // 10 output classes for MNIST

    // Allocate host memory for input and weights
    std::vector<float> input(batch_size * input_size);
    std::vector<float> weights(input_size * output_size);

    // Initialize input and weights with random values
    for (int i = 0; i < batch_size * input_size; ++i) {
        input[i] = static_cast<float>(rand()) / RAND_MAX;
    }
    for (int i = 0; i < input_size * output_size; ++i) {
        weights[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Allocate device memory
    float *d_input, *d_weights, *d_output;
    cudaMalloc(&d_input, batch_size * input_size * sizeof(float));
    cudaMalloc(&d_weights, input_size * output_size * sizeof(float));
    cudaMalloc(&d_output, batch_size * output_size * sizeof(float));

    // Copy input and weights to device
    cudaMemcpy(d_input, input.data(), batch_size * input_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_weights, weights.data(), input_size * output_size * sizeof(float), cudaMemcpyHostToDevice);

    // Create cuBLAS handle
    cublasHandle_t handle;
    cublasCreate(&handle);

    // Perform vector multiplication
    float alpha = 1.0f, beta = 0.0f;
    cublasSgemv(handle, CUBLAS_OP_N, output_size, input_size, &alpha,
                d_weights, output_size, d_input, 1, &beta, d_output, 1);

    // Destroy cuBLAS handle
    cublasDestroy(handle);

    // Copy output back to host
    std::vector<float> output(batch_size * output_size);
    cudaMemcpy(output.data(), d_output, batch_size * output_size * sizeof(float), cudaMemcpyDeviceToHost);

    // Free device memory
    cudaFree(d_input);
    cudaFree(d_weights);
    cudaFree(d_output);

    // Print a sample output
    std::cout << "Sample output:" << std::endl;
    for (int i = 0; i < 10; ++i) {
        std::cout << output[i] << " ";
    }
    std::cout << std::endl;

    return 0;
}


Overwriting cuda_code_1.cu


In [ ]:
!nvcc -o cuda_executable cuda_code_1.cu -lcublas


In [ ]:
!./cuda_executable


Sample output:
66.7555 68.8272 68.0738 61.9798 67.4468 64.2662 68.7616 64.612 60.7305 67.7168 


In [ ]:
import os
import time
import subprocess

# Compile CUDA code
os.system('nvcc -o cuda_executable cuda_code_1.cu')

# Record start time
start_time = time.time()

# Execute compiled CUDA code
os.system('./cuda_executable')

# Record end time
end_time = time.time()

# Calculate execution time
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

# Calculate memory usage
process = subprocess.Popen(['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'],
                           stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, _ = process.communicate()
memory_used = sum(int(x) for x in stdout.strip().split(b'\n'))
print("Memory used:", memory_used, "MiB")


Execution time: 0.001447916030883789 seconds
Memory used: 0 MiB


In [4]:
!wget https://raw.githubusercontent.com/cteqeu/HAC/main/GPU/cuda_profile.cu

--2024-05-07 17:25:22--  https://raw.githubusercontent.com/cteqeu/HAC/main/GPU/cuda_profile.cu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2331 (2.3K) [text/plain]
Saving to: ‘cuda_profile.cu’

cuda_profile.cu     100%[===================>]   2.28K  --.-KB/s    in 0s      

2024-05-07 17:25:22 (54.5 MB/s) - ‘cuda_profile.cu’ saved [2331/2331]



In [5]:
!nvidia-smi

Tue May  7 17:55:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [7]:
!nvcc cuda_profile.cu -o cuda_profile --generate-line-info

In [8]:
!compute-sanitizer ./cuda_profile

========= COMPUTE-SANITIZER
cpu control: 		18108465020928.000000
gpu control: 		18108465020928.000000
gpu um control: 	18108465020928.000000
========= ERROR SUMMARY: 0 errors


In [9]:
!nvprof --print-gpu-trace ./cuda_profile

==11364== NVPROF is profiling process 11364, command: ./cuda_profile
cpu control: 		18108465020928.000000
gpu control: 		18108465020928.000000
gpu um control: 	18108465020928.000000
==11364== Profiling application: ./cuda_profile
==11364== Profiling result:
   Start  Duration            Grid Size      Block Size     Regs*    SSMem*    DSMem*      Size  Throughput  SrcMemType  DstMemType           Device   Context    Stream        Unified Memory  Virtual Address  Name
293.49ms  35.359us                    -               -         -         -         -  390.63KB  10.536GB/s    Pageable      Device     Tesla T4 (0)         1         7                     -                -  [CUDA memcpy HtoD]
293.62ms  34.720us                    -               -         -         -         -  390.63KB  10.730GB/s    Pageable      Device     Tesla T4 (0)         1         7                     -                -  [CUDA memcpy HtoD]
293.98ms  7.4880us             (98 1 1)      (1024 1 1)        16       

In [10]:
!nsys profile cuda_profile

/bin/bash: line 1: nsys: command not found


In [11]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt update
!apt install ./nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt --fix-broken install

--2024-05-07 18:04:53--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 317705436 (303M) [application/x-deb]
Saving to: ‘nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb’

nsight-systems-2023 100%[===================>] 302.99M  18.9MB/s    in 5.0s    

2024-05-07 18:04:58 (60.1 MB/s) - ‘nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb’ saved [317705436/317705436]

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy In

In [12]:
!nsys profile cuda_profile

cpu control: 		18108465020928.000000
gpu control: 		18108465020928.000000
gpu um control: 	18108465020928.000000
Generating '/tmp/nsys-report-bb3f.qdstrm'
[1/1] [========================100%] report1.nsys-rep
Generated:
    /content/report1.nsys-rep


In [13]:
!ncu --set full -o cuda_profile ./cuda_profile

==PROF== Connected to process 19587 (/content/cuda_profile)
cpu control: 		18108465020928.000000
==PROF== Profiling "gpu_testfunction" - 0: 0%....50%....100% - 31 passes
gpu control: 		18108465020928.000000
==PROF== Profiling "gpu_testfunction_uni_mem" - 1: 0%....50%....100% - 31 passes
gpu um control: 	18108465020928.000000
==PROF== Disconnected from process 19587
==PROF== Report: /content/cuda_profile.ncu-rep
